# Machine Learning Final Project

Joey Couse and Abraham Eaton <br>
11-November-2019 <br>

In [1]:
using JuMP, Gurobi, Statistics, DataFrames, LinearAlgebra, StatsBase, CSV, Random

In [2]:
Data=CSV.read("housing.csv",header=true);
Data=convert(Matrix,Data);
Data=Data[shuffle(1:end),:];

In [36]:
selected=findall(x-> x>=0.2 ,abs.(cor(Data)[1:(end-1),end]))
Data[:,selected]

506×12 Array{Float64,2}:
  0.09604  40.0   6.41  0.447  6.854  …   4.0  254.0  17.6  396.9    2.98
  0.11432   0.0   8.56  0.52   6.781      5.0  384.0  20.9  395.58   7.67
  0.04544   0.0   3.24  0.46   6.144      4.0  430.0  16.9  368.57   9.09
  0.04011  80.0   1.52  0.404  7.287      2.0  329.0  12.6  396.9    4.08
  0.55007  20.0   3.97  0.647  7.206      5.0  264.0  13.0  387.89   8.1 
  0.07896   0.0  12.83  0.437  6.273  …   5.0  398.0  18.7  394.92   6.78
  0.52693   0.0   6.2   0.504  8.725      8.0  307.0  17.4  382.0    4.63
 18.0846    0.0  18.1   0.679  6.434     24.0  666.0  20.2   27.25  29.05
 23.6482    0.0  18.1   0.671  6.38      24.0  666.0  20.2  396.9   23.69
  0.04666  80.0   1.52  0.404  7.107      2.0  329.0  12.6  354.31   8.61
 13.6781    0.0  18.1   0.74   5.935  …  24.0  666.0  20.2   68.95  34.02
  0.09378  12.5   7.87  0.524  5.889      5.0  311.0  15.2  390.5   15.71
  4.75237   0.0  18.1   0.713  6.525     24.0  666.0  20.2   50.92  18.13
  ⋮          

In [37]:
selected

12-element Array{Int64,1}:
  1
  2
  3
  5
  6
  7
  8
  9
 10
 11
 12
 13

In [ ]:
n=size(Data,1);
print("Number of observations: ")
println(n);
m=size(Data,2);
println();

X=Data[:,1:(m-1)];
Y=Data[:,m];
test_X=Test_Data[:,1:(m-1)];
test_Y=Test_Data[:,m];


m=m-1;
print("Number of Features: ")
println(m);
println();
print("Percent of data in validation set: ")
ρ=0.25;
println(ρ);

In [8]:


Balanced = Model(solver=GurobiSolver(TimeLimit=90));

@variable(Balanced, z[1:n], Bin);
@constraint(Balanced, sum(z[:])==floor(ρ*n));

@objective(Balanced, Min, sum(1/floor(ρ*n)*sum(z[i]*X[i,j] for i = 1:n) - 
    1/ceil((1-ρ)*n)*sum((1-z[i])*X[i,j] for i = 1:n) for j = 1:m));

solve(Balanced);

In [48]:
test_obs_count=round(Int,0.1*size(Data,1));
Test_Data = Data[1:test_obs_count,:];
Data=Data[(test_obs_count+1):end,:];

In [35]:
sum(getvalue(z));

113.0

In [90]:
k= (1-ρ)*n; #number of observations in training set
λ= 1; #regularization constant

Stable = Model(solver=GurobiSolver(TimeLimit=90));

@variable(Stable, B[1:m]);
@variable(Stable, θ);
@variable(Stable, μ[1:n] >=0);

@constraint(Stable, [i=1:n], θ+μ[i] >= Y[i]-sum(X[i,:].*B));
@constraint(Stable, [i=1:n], θ+μ[i] >= -Y[i]+sum(X[i,:].*B));

@objective(Stable, Min, k*θ+sum(μ[i] for i = 1:n)+λ*sum(B.^2));

solve(Stable);

In [91]:
Betas=getvalue(B);

In [92]:
MSE=mean((sum(Betas.*test_X[i,:]) .- test_Y[i]).^2 for i = 1:size(test_X,1))

22.22200390597508

In [63]:
SVB = function(Data,λ0=0,granularity=1,runs=10,print_results=true,seed=rand(1:2019),percent_validation=0.25,Y_col_last=true,test_percentage=0.1)
    
    if print_results==true
        print("Seed: ")
        println(seed);
        println();
        println("-------------------------------------------------------------")
        println();
    end
    
    function LASSO_Regression(X,Y,tun)
        features=size(X,2);
        obs=size(Y,1);
        #M=10000;
        Tun=tun;
        LASSO = Model(solver=GurobiSolver(TimeLimit=30))
        @variable(LASSO, Bn[1:features]);
        #@variable(LASSO, z1[1:features], Bin);
        #@constraint(LASSO, M*z1.>= -Bn);
        #@constraint(LASSO, M*z1.>=  Bn);
        #@constraint(LASSO, sum(z1) <= k );
        #@objective(LASSO, Min, sum((Y-X*Bn).^2)+tun*(sum(z1)));
        @objective(LASSO,Min,sum((Y-X*Bn).^2)+tun*sum(Bn.^2));
        
        redirect_stdout((()->solve(LASSO)),open("C:\\Users\\joeyc\\Documents\\gurobi_output","w"))
        Bs=getvalue(Bn)
        return Bs
    end
    
    
    
    ρ=percent_validation;
    
    ##
    ##
    ##
    ## Data Pre-Processing
    ##
    ##
    ##
    
    Data=convert(Matrix,Data);
    Random.seed!(seed);
    Data=Data[shuffle(1:end),:];
    
    # Normalizing Data
    for j in 1:size(Data,2)
        mn = mean(Data[:,j])
        sd = std(Data[:,j])
        for i in 1:size(Data,1)
            Data[i,j]=Data[i,j].-mn./sd
        end
    end
    
    test_obs_count=round(Int,test_percentage*size(Data,1));
    Test_Data = Data[1:test_obs_count,:];
    Data=Data[(test_obs_count+1):end,:];
    
    
    
    
    n=size(Data,1);
    m=size(Data,2);
    
    if (Y_col_last == true)
        X=Data[:,1:(m-1)];
        Y=Data[:,m];
        test_X=Test_Data[:,1:(m-1)];
        test_Y=Test_Data[:,m];
    else
        X=Data[:,2:(m)];
        Y=Data[:,1];
        test_X=Test_Data[:,2:(m)];
        test_Y=Test_Data[:,1];
    end
    m=m-1;
    #ρ=0.25;
    
    ##
    ##
    ##
    ##
    ## Balanced Regression
    ##
    ##
    ##
    ##
    if print_results==true
        println("Balanced Regression Results:");
        println();
    end
    
    Balanced = Model(solver=GurobiSolver(TimeLimit=90));

    @variable(Balanced, z[1:n], Bin);
    @constraint(Balanced, sum(z[:])==floor(ρ*n));

    @objective(Balanced, Min, sum(1/floor(ρ*n)*sum(z[i]*X[i,j] for i = 1:n) - 
        1/ceil((1-ρ)*n)*sum((1-z[i])*X[i,j] for i = 1:n) for j = 1:m)^2);

    redirect_stdout((()->solve(Balanced)),open("C:\\Users\\joeyc\\Documents\\gurobi_output","w"));

    ###
    ###
    ###
    ###
    ### Do Lasso stuff here
    ###
    ###
    ###
    ###
    
    Betas_Mat=zeros(runs,m);
    λval=zeros(runs);
    
    Is_Validation=round.(Int,getvalue(z));
    

    for i = 1:runs
        λ = (i-1)*granularity+λ0;
        Betas_Mat[i,:]=LASSO_Regression(X[Is_Validation.==0,:],Y[Is_Validation.==0,:],λ);
        λval[i]=λ;
    end
    
    valid_X=X[Is_Validation.==1,:];
    valid_Y=Y[Is_Validation.==1,:];
    
    MSE_Best_Balanced=mean((sum(Betas_Mat[1,:].*valid_X[i,:]) .- valid_Y[i]).^2 for i = 1:size(valid_X,1));
    λbest=λval[1];
    β_Best_Balanced=Betas_Mat[1,:];
    
    
    for j = 1:runs
        MSE = mean((sum(Betas_Mat[j,:].*valid_X[i,:]) .- valid_Y[i]).^2 for i = 1:size(valid_X,1));
        if (MSE <= MSE_Best_Balanced)
            MSE_Best_Balanced = MSE;
            λbest=λval[j];
            β_Best_Balanced=Betas_Mat[j,:];
        end
    end
    
    if print_results==true
        println("Best Balanced Lambda:");
        println(λbest);
        println();
        println("Best Balanced Validation MSE: ");
        println(MSE_Best_Balanced);
        println();
    end
    
    β_Best_Balanced=LASSO_Regression(X,Y,λbest);
    
    MSE_Best_Balanced=mean((sum(β_Best_Balanced.*test_X[i,:]) .- test_Y[i]).^2 for i = 1:size(test_X,1));
    
    if print_results==true
        println("Test MSE for Balanced Regression:")
        println(MSE_Best_Balanced);
        println();
        println("-------------------------------------------------------------")
    end
    
    ###
    ###
    ###
    ###
    ### This is where Stable Regression starts
    ###
    ###
    ###
    ###
    
    train_obs_count= (1-ρ)*n; #number of observations in training set
    λ= 1; #regularization constant
    
    Stable_Regression = function(X,Y,k,λ)
        m=size(X,2);
        n=size(Y,1);
        
        Stable = Model(solver=GurobiSolver(TimeLimit=90));

        @variable(Stable, B[1:m]);
        @variable(Stable, θ);
        @variable(Stable, μ[1:n] >=0);

        @constraint(Stable, [i=1:n], θ+μ[i] >= Y[i]-sum(X[i,:].*B));
        @constraint(Stable, [i=1:n], θ+μ[i] >= -Y[i]+sum(X[i,:].*B));

        @objective(Stable, Min, k*θ+sum(μ[i] for i = 1:n)+λ*sum(B.^2));
        redirect_stdout((()->solve(Stable)),open("C:\\Users\\joeyc\\Documents\\gurobi_output","w"));
        Betas_Stable=getvalue(B);
        Is_Train=Int.(getvalue(μ).>=0.0001);
        #Is_Train=getvalue(μ);
        return Betas_Stable, Is_Train
    end
    
    if print_results==true
        println("Stable Regression Results:");
        println();
    end
    
    Betas_Mat=zeros(runs,m);
    λval=zeros(runs);
    Is_Train_Mat=zeros(size(X,1),runs);

    for i = 1:runs
        λ = (i-1)*granularity+λ0;
        res=Stable_Regression(X,Y,train_obs_count,λ);
        Betas_Mat[i,:]=res[1];
        Is_Train_Mat[:,i]=res[2];
        λval[i]=λ;
    end
    
    Stable_Valid_MSE=10000;
    λbest_Stable=-100;
    Best_Betas_Stable=zeros(size(Betas_Mat,2));
    
    for j = 1:runs
        valid_X=X[Is_Train_Mat[:,j].==0,:];
        valid_Y=Y[Is_Train_Mat[:,j].==0,:];
        MSE = mean((sum(Betas_Mat[j,:].*valid_X[i,:]) .- valid_Y[i]).^2 for i = 1:size(valid_X,1));
        
        if (MSE <= Stable_Valid_MSE)
            Stable_Valid_MSE=MSE;
            λbest_Stable=λval[j];
            Best_Betas_Stable=Betas_Mat[j,:];
        end
    end
    
    if print_results==true
        println("Best Stable Lambda: ");
        println(λbest_Stable);
        println();
        println("Best Stable Validation MSE: ");
        println(Stable_Valid_MSE);
        println();
    end
    
    MSE_Best_Stable=mean((sum(Best_Betas_Stable.*test_X[i,:]) .- test_Y[i]).^2 for i = 1:size(test_X,1));
    
    if print_results==true
        println("Test MSE for Stable Regression:")
        println(MSE_Best_Stable);
        println();
        println("-------------------------------------------------------------")
    end
    
    return MSE_Best_Balanced, MSE_Best_Stable
end

#129 (generic function with 9 methods)

In [ ]:
wins=0
for i in 1:1000     
    (Balance), (Stable) =SVB(Data, 0, 0.05,3, false)
    if (Balance<Stable)
        wins=wins+1
    end
end

In [ ]:
wins